# Video Summarization Steps
### Convert frames to base64
### Get audio content
### Summarize the video only
### summarize the audio only
### summarize both audio and video together

In [ ]:
%pip install --upgrade openai --quiet
%pip install opencv-python --quiet
%pip install moviepy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
open_ai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI
import os

model = "gpt-4o"
client = OpenAI(api_key= open_ai_api_key)

In [ ]:
import cv2
from moviepy.editor import VideoFileClip
import time
import base64

In [ ]:
VIDEO_PATH = "/content/keynote_recap.mp4"

In [ ]:
def process_video(video_path, seconds_per_frame =2 ):
  base64frames = []
  base_video_path,_ = os.path.splitext(video_path)
  print(base_video_path)

  video = cv2.VideoCapture(video_path)
  total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = video.get(cv2.CAP_PROP_FPS)
  frames_to_skip = int(fps*seconds_per_frame)

  curr_frame = 0

  while curr_frame < total_frames - 1:
    video.set(cv2.CAP_PROP_POS_FRAMES, curr_frame)
    sucess, frame = video.read()

    if not sucess:
      break

    _, buffer = cv2.imencode(".jpg", frame)
    base64frames.append(base64.b64encode(buffer).decode("utf-8"))
    curr_frame = curr_frame + frames_to_skip

  video.release()

  audio_path = f"{base_video_path}.mp3"
  clip = VideoFileClip(video_path)
  clip.audio.write_audiofile(audio_path, bitrate="32k")
  clip.audio.close()
  clip.close()

  return base64frames, audio_path









In [ ]:
base64_frames, audio_path = process_video(VIDEO_PATH)

/content/keynote_recap
MoviePy - Writing audio in /content/keynote_recap.mp3


MoviePy - Done.


In [ ]:
base64_frames = base64_frames[::20]

In [ ]:
respose = client.chat.completions.create(
    model = model,
    messages = [
        {
            "role": "system",
            "content": "You are generating a video summary, Please provide a summary of the video, repond in Markdown"
        },
        {
            "role" : "user",
            "content": [
                "There are the frames from the video",
                *map(lambda x: {"type": "image_url",
                                "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}

                }, base64_frames)
            ]
        }
    ],
    temperature = 0,
)

In [ ]:
print(respose.choices[0].message.content)

### Video Summary

The video appears to be a presentation from an event called "OpenAI DevDay." The presentation includes the following key points:

1. **Introduction Slide**: The video starts with a title slide displaying "OpenAI DevDay."
2. **Function Calling Slide**: A slide explaining the concept of function calling, showing a "before" and "after" scenario of how commands are processed.
3. **Presenter**: A person is seen speaking on stage, likely the main presenter of the event.
4. **Key Topics Slide**: The presentation covers three main topics: Instructions, Expanded Knowledge, and Actions.
5. **Presenter's Speech**: The presenter continues to elaborate on the topics, engaging with the audience.
6. **Closing Gesture**: The presenter waves, possibly indicating the end of the presentation or a segment.

The video seems to focus on advancements in AI, particularly in how functions are called and executed, and the broader implications of these advancements.
